<a href="https://colab.research.google.com/github/mzwing/AI-related/blob/master/notebooks/stable-diffusion.cpp/Hassaku_GGUF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prepare
!rm -rf sample_data
!mkdir -p Hassaku-GGUF

In [ ]:
!pip install -U --force-reinstall huggingface_hub[hf_transfer]

In [ ]:
!huggingface-cli login

In [ ]:
# get original model
!sudo apt-get install aria2 -y
!mkdir -p Hassaku
!echo -e "https://civitai.com/api/download/models/106922?type=Model&format=SafeTensor&size=pruned&fp=fp16\n out=hassakuHentaiModel_v13.safetensors\nhttps://civitai.com/api/download/models/62528\n out=hassakuHentaiModel_v12.safetensors\nhttps://civitai.com/api/download/models/37521\n out=hassakuHentaiModel_v11.safetensors\nhttps://civitai.com/api/download/models/32763\n out=hassakuHentaiModel_v1m.safetensors\nhttps://civitai.com/api/download/models/29539\n out=hassakuHentaiModel_v1.safetensors\nhttps://civitai.com/api/download/models/30191\n out=hassakuHentaiModel_v1to13Inpainting.safetensors" > download.txt
!aria2c -c -x16 -d Hassaku --input-file=download.txt
!rm -rf download.txt

In [ ]:
# get compile result (CPU)
!aria2c -c -x16 https://github.com/MZWNET/actions/releases/download/sd-master-583cc5b/sd-master-583cc5b-bin-linux-avx2-openblas-x64.zip
!unzip sd-master-583cc5b-bin-linux-avx2-openblas-x64.zip -d .
!rm -rf sd-master-583cc5b-bin-linux-avx2-openblas-x64.zip

In [ ]:
# prepare for GPU
!sudo apt-get install nvidia-cuda-toolkit -y

In [ ]:
# get compile result (GPU)
!aria2c -c -x16 https://github.com/MZWNET/actions/releases/download/sd-master-583cc5b/sd-master-583cc5b-bin-linux-avx2-cublas-cu121-x64.zip
!unzip sd-master-583cc5b-bin-linux-avx2-cublas-cu121-x64.zip -d .
!rm -rf sd-master-583cc5b-bin-linux-avx2-cublas-cu121-x64.zip

In [ ]:
# quantize
import concurrent.futures
import os
from tqdm import tqdm

parameters = [ "f16", "q8_0", "q5_1", "q5_0", "q4_1", "q4_0" ]
models = [ "hassakuHentaiModel_v13.safetensors", "hassakuHentaiModel_v12.safetensors", "hassakuHentaiModel_v11.safetensors", "hassakuHentaiModel_v11m.safetensors", "hassakuHentaiModel_v1.safetensors", "hassakuHentaiModel_v1to13Inpainting.safetensors" ]

tasks = [(model, param) for model in models for param in parameters]

def run_command(task):
    model, param = task
    model_name = model.split('.')[0]
    os.system(f"./sd -M convert -m Hassaku/{model} -o Hassaku-GGUF/{model_name}.{param}.gguf --type {param}")

with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    list(tqdm(executor.map(run_command, tasks), total=len(tasks)))

In [ ]:
# test
!./sd -m Hassaku-GGUF/hassakuHentaiModel_v11.q4_0.gguf -p "1girl, extremely beautiful, miniskirt" -s -1 -v -o test.png

In [ ]:
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli upload --repo-type model --commit-message "GGUF model commit (made with stable-diffusion.cpp commit 583cc5b)" Hassaku-GGUF ./Hassaku-GGUF